In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report,confusion_matrix,plot_confusion_matrix,ConfusionMatrixDisplay

In [2]:
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 25

In [3]:
with open("fer2013.csv") as f:
  content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size
print("number of instances: ",num_of_instances)
print("instance length: ",len(lines[3].split(",")[1].split(" ")))

number of instances:  35888
instance length:  2304


In [65]:
x_train, y_train, x_test, y_test = [], [], [], []

In [66]:
for i in range(1,num_of_instances):
        emotion, img, usage = lines[i].split(",")
          
        val = img.split(" ")
            
        pixels = np.array(val, 'float32')    
        if 'Training'in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        else:
            y_test.append(emotion)
            x_test.append(pixels)    

In [67]:
y_test

['0',
 '1',
 '4',
 '6',
 '3',
 '3',
 '2',
 '0',
 '2',
 '0',
 '3',
 '0',
 '4',
 '2',
 '6',
 '2',
 '5',
 '0',
 '5',
 '3',
 '2',
 '5',
 '0',
 '4',
 '0',
 '2',
 '4',
 '4',
 '0',
 '2',
 '3',
 '6',
 '3',
 '6',
 '0',
 '4',
 '6',
 '2',
 '6',
 '3',
 '0',
 '4',
 '4',
 '2',
 '6',
 '3',
 '5',
 '3',
 '5',
 '6',
 '0',
 '3',
 '5',
 '4',
 '4',
 '3',
 '4',
 '2',
 '5',
 '3',
 '6',
 '3',
 '4',
 '3',
 '5',
 '3',
 '5',
 '4',
 '2',
 '6',
 '0',
 '5',
 '6',
 '6',
 '2',
 '6',
 '5',
 '6',
 '4',
 '4',
 '0',
 '0',
 '1',
 '3',
 '5',
 '4',
 '3',
 '3',
 '3',
 '3',
 '6',
 '0',
 '6',
 '4',
 '5',
 '2',
 '3',
 '3',
 '3',
 '0',
 '3',
 '6',
 '0',
 '3',
 '2',
 '3',
 '6',
 '6',
 '5',
 '3',
 '3',
 '0',
 '0',
 '2',
 '3',
 '2',
 '3',
 '6',
 '2',
 '6',
 '5',
 '0',
 '0',
 '2',
 '4',
 '6',
 '3',
 '6',
 '5',
 '5',
 '2',
 '0',
 '2',
 '6',
 '5',
 '5',
 '5',
 '6',
 '2',
 '6',
 '6',
 '3',
 '1',
 '4',
 '4',
 '6',
 '3',
 '3',
 '3',
 '2',
 '2',
 '0',
 '5',
 '3',
 '5',
 '5',
 '4',
 '5',
 '0',
 '3',
 '0',
 '3',
 '3',
 '0',
 '6',
 '6',
 '3'

In [68]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test= tf.keras.utils.to_categorical(y_test, num_classes)

In [69]:
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')
x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

28709 train samples
7178 test samples


In [70]:
y_test


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [17]:
model = Sequential()

#1st convolution layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48,48,1)))
#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(strides=(2, 2)))
model.add(Dropout(0.1))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(strides=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(strides=(2, 2)))
model.add(Dropout(0.1))
model.add(Flatten())

#fully connected neural networks
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [18]:
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)
test_generator=gen.flow(x_test,y_test,batch_size=batch_size)

In [ ]:
model.compile(loss='categorical_crossentropy'
    , optimizer='adam'
    , metrics=['accuracy']
)

In [ ]:
hist=model.fit(train_generator, epochs=epochs,validation_data=(test_generator)) #train for all trainset


In [ ]:
hist=model.fit(train_generator, epochs=10,validation_data=(test_generator)) #train for all trainset


In [ ]:
model.save('model25.h5')

In [10]:
from keras.models import load_model
model = load_model('model25.h5')

In [64]:
train_score = model.evaluate(x_train, y_train, verbose=1)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = model.evaluate(x_test,y_test, verbose=1)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

156/156 [==============================] - 9s 57ms/step - loss: 0.1747 - accuracy: 0.9535
Train loss: 0.17472721636295319
Train accuracy: 95.34743428230286
39/39 [==============================] - 2s 56ms/step - loss: 1.3739 - accuracy: 0.6318
Test loss: 1.3739088773727417
Test accuracy: 63.17923665046692


In [23]:
def emotion_analysis(emotions):
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))
    
    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
    plt.show()

In [ ]:

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt


file ='/content/5.jpg'
true_image = image.load_img(file)
img = image.load_img(file, color_mode='grayscale', target_size=(48, 48))

x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48]);

plt.gray()
plt.imshow(true_image)
plt.show()

In [ ]:
*	model.fit(x_train,y_train, epochs=epochs,batch_size=batch_size,validation_data=(x_test,y_test)) #train for all trainset


In [26]:
import matplotlib.pyplot as plt

metrics = hist.history
plt.plot(hist.epoch, metrics['loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

NameError: name 'hist' is not defined

In [ ]:
target_names=['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
labels_names=[0,1,2,3,4,5,6]
Y_pred=model.predict_generator(x_test)
y_pred=np.argmax(Y_pred,axis)
cn=confusion_matrix(y_test.classes,y_pred,labels=labels_names)

print('Confusion Matrix')

print(confusion_ma)

In [71]:
from sklearn.metrics import classification_report, confusion_matrix
 
pred_list = []; actual_list = []
predictions = model.predict(x_test)

for i in predictions:
 
  pred_list.append(np.argmax(i))
 
for i in y_test:
 
  actual_list.append(np.argmax(i))
 
confusion_matrix(actual_list, pred_list)
#classification_report(actual_list,pred_list)

array([[ 556,   13,   87,   40,  122,   22,  118],
       [  24,   58,   10,    3,    9,    1,    6],
       [ 129,    6,  492,   30,  174,   73,  120],
       [  63,    1,   38, 1482,   54,   47,   89],
       [ 156,    3,  157,   79,  616,   21,  215],
       [  27,    1,   59,   37,   20,  656,   31],
       [  76,    2,   79,   98,  173,   26,  779]], dtype=int64)

In [72]:
print(classification_report(actual_list,pred_list))

              precision    recall  f1-score   support

           0       0.54      0.58      0.56       958
           1       0.69      0.52      0.59       111
           2       0.53      0.48      0.51      1024
           3       0.84      0.84      0.84      1774
           4       0.53      0.49      0.51      1247
           5       0.78      0.79      0.78       831
           6       0.57      0.63      0.60      1233

    accuracy                           0.65      7178
   macro avg       0.64      0.62      0.63      7178
weighted avg       0.65      0.65      0.65      7178



In [ ]:
monitor_testset_results = True

if monitor_testset_results == True:
	#make predictions for test set
	predictions = model.predict(x_test)

	index = 0
	for i in predictions:
		if index < 30 and index >= 20:
			#print(i) #predicted scores
			#print(y_test[index]) #actual scores
			
			testing_img = np.array(x_test[index], 'float32')
			testing_img = testing_img.reshape([48, 48]);
			
			plt.gray()
			plt.imshow(testing_img)
			plt.show()
			
			print(i)
			
			emotion_analysis(i)
			print("----------------------------------------------")
		index = index + 1